In [ ]:
import pandas as pd
import nltk
import spacy
import matplotlib.pyplot as plt
import numpy as np
import string

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lucasmoratof/customers_review_project/master/reviews_for_nlp.csv',
                 index_col=0)
df = df.rename(columns={'review_comment_message':'text', 'is_good_review':'sentiment'})

In [ ]:
def remove_pontuacao(text):
    # Remover todas as partes que forem pontuação
    clean_text = text.translate(str.maketrans('', '', string.punctuation))
    return clean_text

def para_minuscula(text):
    # Transforma todo o texto para minúsculas
    return text.lower()

def remove_line_break(text):
    # Remove quebras de linha
    return text.replace('\n', '')

def preprocessamento(text):
    # Faz todo o pré-processamento do texto
    texto_minusculo = para_minuscula(text)
    no_line_break_text = remove_line_break(texto_minusculo)
    clean_text = remove_pontuacao(no_line_break_text)
    return clean_text

In [ ]:
df['clean_text'] = df['text'].apply(preprocessamento)

In [ ]:
df['text'].head()

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df['clean_text']

In [ ]:
#!python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

def lematizar(texto_original):
    # Rodando lematização nas palavras
    doc = nlp(texto_original)
    
    doc_lematizado = [token.lemma_ if token.pos_ in ('VERB', 'AUX') else str(token) for token in doc]

    return ' '.join(doc_lematizado)

In [ ]:
%%time
# A lematização é um bom passo de pré-processamento, porém ela demora muito.
df['lematized_text'] = df['clean_text'].apply(lematizar)

In [ ]:
df['lematized_text'].head()

In [ ]:
#Agora que o texto já foi pré-processado, podemos usar TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = df['lematized_text']
y = df['sentiment']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Nota que embora o pré-processamento é feito antes, visto que ele não usa nenhum aprendizado no treino,
# o TF-IDF aprende o vocabulário no treino. Então NÃO POSSO usar "fit" na base de validação.
train_matrix = vectorizer.fit_transform(X_train)
test_matrix = vectorizer.transform(X_test)

In [ ]:
%%time
# O SVM também é um modelo que costuma demorar para treinar quando temos muitos dados.
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(train_matrix, y_train)
predictions = model_svm.predict(test_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix
y_test_array = np.asarray(y_test)
confusion_matrix(y_test, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))